# Configuration

In [1]:
# Parameters
PROJECT_NAME = 'ML1010-Group-Project'
ENABLE_COLAB = False

#Root Machine Learning Directory. Projects appear underneath
GOOGLE_DRIVE_MOUNT = '/content/gdrive' 
COLAB_ROOT_DIR = GOOGLE_DRIVE_MOUNT + '/MyDrive/Colab Notebooks'
COLAB_INIT_DIR = COLAB_ROOT_DIR + '/utility_files'

LOCAL_ROOT_DIR = '/home/magni/Documents/ML_Projects'
LOCAL_INIT_DIR = LOCAL_ROOT_DIR + '/utility_files'

# Bootstrap Environment

In [2]:
#add in support for utility file directory and importing
import sys
import os

if ENABLE_COLAB:
  #Need access to drive
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE_MOUNT, force_remount=True)
  
  #add in utility directory to syspath to import 
  INIT_DIR = COLAB_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = COLAB_ROOT_DIR
  
else:
  #add in utility directory to syspath to import
  INIT_DIR = LOCAL_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = LOCAL_ROOT_DIR

#Import Utility Support
from jarvis import Jarvis
jarvis = Jarvis(ROOT_DIR, PROJECT_NAME)



import mv_python_utils as mvutils


Mounted at /content/gdrive
Wha...where am I?
I am awake now.

I have set your current working directory to /content/gdrive/MyDrive/Colab Notebooks/ML1010-Group-Project
The current time is 09:56
Hello sir. Extra caffeine may help.



# Setup Runtime Environment

In [3]:
#%%capture
if ENABLE_COLAB:
  #!pip install scipy -q
  #!pip install scikit-learn -q
  #!pip install pycaret -q
  #!pip install matplotlib -q
  #!pip install joblib -q
  #!pip install pandasql -q 
  !pip install sentence_transformers -q
  !pip install flair -q
  !pip install spacy -U -q
  !pip install spacytextblob -q
  display('Google Colab enabled')
else:
  display('Google Colab not enabled')

#Common imports
import json
import gzip
import pandas as pd
import numpy as np
import matplotlib
import re
import nltk
import matplotlib.pyplot as plt

!python -m spacy download en_core_web_sm

nltk.download('stopwords')
%matplotlib inline

import cw_df_metric_utils as cwutils

     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 3.1 MB 11.6 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
     |████████████████████████████████| 1.2 MB 32.1 MB/s 
     |████████████████████████████████| 61 kB 483 kB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 
     |████████████████████████████████| 322 kB 5.1 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 788 kB 39.8 MB/s 
     |████████████████████████████████| 19.7 MB 1.2 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 1.2 MB 56.7 MB/s 
     |████████████████████████████████| 981 kB 65.6 MB/s 
     |████████████████████████████████| 62 kB 750 kB/s 
ERROR: pip's de

'Google Colab enabled'

     |████████████████████████████████| 13.9 MB 1.6 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load Data

In [4]:
### load the meta data
jarvis.showProjectDataFiles()
data = []
with gzip.open(jarvis.DATA_DIR + '/01_original/Cell_Phones_and_Accessories_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

Here are all your project data files
[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project
----->** No files **

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/01_original
---[   gz][ json]--> Cell_Phones_and_Accessories_5.json.gz (161.24 MB)
---[   gz][ json]--> meta_Cell_Phones_and_Accessories.json.gz (343.33 MB)

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/02_working
[*][  pkl]---------> 02_Cellphone.pkl (46.32 MB)
[*][  pkl]---------> 02_NLP_ReviewTextData.pkl (87.00 MB)
[*][  pkl]---------> 02_NLP_SummaryData.pkl (8.32 MB)
[*][  pkl]---------> 02_NLP_TitleData.pkl (16.71 MB)
[*][  pkl]---------> 03_Cellphone.pkl (46.31 MB)
[*][  pkl]---------> 03_NLP_ReviewTextData.pkl (28.94 MB)
[*][  pkl]---------> 03_NLP_ReviewText_Narrow.pkl (17.13 MB)
[*][  pkl]---------> 03_NLP_SummaryData.pkl (3.82 MB)
[*][  pkl]---------> 03_NLP_TitleData.pkl (2.73 MB)
[*][  pkl]---------> 04_NLP_ReviewText_Narrow.pkl (16.95 MB)
[*][  pkl]---

In [5]:
# convert list into pandas dataframe
df = pd.DataFrame.from_dict(data)

print(len(df))

1128437


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128437 entries, 0 to 1128436
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1128437 non-null  float64
 1   verified        1128437 non-null  bool   
 2   reviewTime      1128437 non-null  object 
 3   reviewerID      1128437 non-null  object 
 4   asin            1128437 non-null  object 
 5   style           605241 non-null   object 
 6   reviewerName    1128302 non-null  object 
 7   reviewText      1127672 non-null  object 
 8   summary         1127920 non-null  object 
 9   unixReviewTime  1128437 non-null  int64  
 10  vote            92034 non-null    object 
 11  image           27107 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 95.8+ MB


In [7]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'], format="%m %d, %Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128437 entries, 0 to 1128436
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   overall         1128437 non-null  float64       
 1   verified        1128437 non-null  bool          
 2   reviewTime      1128437 non-null  datetime64[ns]
 3   reviewerID      1128437 non-null  object        
 4   asin            1128437 non-null  object        
 5   style           605241 non-null   object        
 6   reviewerName    1128302 non-null  object        
 7   reviewText      1127672 non-null  object        
 8   summary         1127920 non-null  object        
 9   unixReviewTime  1128437 non-null  int64         
 10  vote            92034 non-null    object        
 11  image           27107 non-null    object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 95.8+ MB


In [8]:
df.head(5)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,2014-08-04,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5.0,True,2014-02-12,A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,3.0,True,2014-02-08,AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,2.0,True,2014-02-04,A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4.0,True,2014-02-03,A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN


In [9]:
### load the meta data
dataMeta = []
with gzip.open(jarvis.DATA_DIR + '/01_original/meta_Cell_Phones_and_Accessories.json.gz') as f:
    for l in f:
        dataMeta.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(dataMeta))

# first row of the list
print(dataMeta[0])

590071
{'category': ['Cell Phones & Accessories', 'Accessories', 'Screen Protectors'], 'tech1': '', 'description': [], 'fit': '', 'title': 'Trollies Sing a Long Party VHS', 'also_buy': [], 'tech2': '', 'brand': '', 'feature': [], 'rank': '557,379 in Movies & TV (', 'also_view': [], 'details': {'ASIN: ': '6303195164'}, 'main_cat': 'Movies & TV', 'similar_item': '', 'date': '', 'price': '', 'asin': '6303195164', 'imageURL': [], 'imageURLHighRes': []}


In [10]:
# convert list into pandas dataframe
dfMeta = pd.DataFrame.from_dict(dataMeta)

print(len(dfMeta))

590071


In [11]:
dfMeta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590071 entries, 0 to 590070
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   category         590071 non-null  object
 1   tech1            590071 non-null  object
 2   description      590071 non-null  object
 3   fit              590071 non-null  object
 4   title            590071 non-null  object
 5   also_buy         590071 non-null  object
 6   tech2            590071 non-null  object
 7   brand            590071 non-null  object
 8   feature          590071 non-null  object
 9   rank             590071 non-null  object
 10  also_view        590071 non-null  object
 11  details          590071 non-null  object
 12  main_cat         590071 non-null  object
 13  similar_item     590071 non-null  object
 14  date             590071 non-null  object
 15  price            590071 non-null  object
 16  asin             590071 non-null  object
 17  imageURL  

In [12]:
dfMeta.head(5)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,"[Cell Phones & Accessories, Accessories, Scree...",,[],,Trollies Sing a Long Party VHS,[],,,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,,,,6303195164,[],[]
1,"[Cell Phones & Accessories, Accessories, Car A...",,[],,La Pura VHS,[],,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,,,,630524412X,[],[]
2,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[New items to give you a new good mood, welcom...",,Leopard Thanksgiving Easter PC Black Case for ...,[],,,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,,,,6310200240,[],[]
3,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[A brand-new, unused, unopened, undamaged item...",,Puppies Faceplate Hard Case Protector for Net1...,[],,Live2Pedal,[Protect your phone with style through this At...,"[>#6,368,825 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,6786501710,[],[]
4,"[Cell Phones & Accessories, Cases, Holsters & ...",,"[A brand-new, unused, unopened, undamaged item...",,White Wolf Faceplate Protector Hard Case for S...,[],,wirelessoutletusa,[Protect your phone with style through this At...,"[>#2,672,454 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,6786501273,[],[]


In [13]:
dfMerge = pd.merge(df, dfMeta, how="inner", on=["asin"])

In [14]:
dfMerge.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1129035 entries, 0 to 1129034
Data columns (total 30 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   overall          1129035 non-null  float64       
 1   verified         1129035 non-null  bool          
 2   reviewTime       1129035 non-null  datetime64[ns]
 3   reviewerID       1129035 non-null  object        
 4   asin             1129035 non-null  object        
 5   style            605459 non-null   object        
 6   reviewerName     1128900 non-null  object        
 7   reviewText       1128270 non-null  object        
 8   summary          1128518 non-null  object        
 9   unixReviewTime   1129035 non-null  int64         
 10  vote             92176 non-null    object        
 11  image            27106 non-null    object        
 12  category         1129035 non-null  object        
 13  tech1            1129035 non-null  object        
 14  de

In [15]:
dfMerge['category'] = dfMerge['category'].astype("string")
#dfMerge['style'] = dfMerge['style'].astype("string")
dfMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1129035 entries, 0 to 1129034
Data columns (total 30 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   overall          1129035 non-null  float64       
 1   verified         1129035 non-null  bool          
 2   reviewTime       1129035 non-null  datetime64[ns]
 3   reviewerID       1129035 non-null  object        
 4   asin             1129035 non-null  object        
 5   style            605459 non-null   object        
 6   reviewerName     1128900 non-null  object        
 7   reviewText       1128270 non-null  object        
 8   summary          1128518 non-null  object        
 9   unixReviewTime   1129035 non-null  int64         
 10  vote             92176 non-null    object        
 11  image            27106 non-null    object        
 12  category         1129035 non-null  string        
 13  tech1            1129035 non-null  object        
 14  de

In [16]:
dfMerge.head(5)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,imageURL,imageURLHighRes
0,5.0,True,2014-08-04,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN,"['Cell Phones & Accessories', 'Cases, Holsters...",,"[, Elegani Butterfly Case 3D Pattern Back Cove...",,MinisDesign 3d Bling Crystal Bow Transparent C...,[],,ELEGANI,[Fashionable with unique 3D butterfly design f...,"[>#228,544 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,5.0,True,2014-02-12,A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN,"['Cell Phones & Accessories', 'Cases, Holsters...",,"[, Elegani Butterfly Case 3D Pattern Back Cove...",,MinisDesign 3d Bling Crystal Bow Transparent C...,[],,ELEGANI,[Fashionable with unique 3D butterfly design f...,"[>#228,544 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,3.0,True,2014-02-08,AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN,"['Cell Phones & Accessories', 'Cases, Holsters...",,"[, Elegani Butterfly Case 3D Pattern Back Cove...",,MinisDesign 3d Bling Crystal Bow Transparent C...,[],,ELEGANI,[Fashionable with unique 3D butterfly design f...,"[>#228,544 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,2.0,True,2014-02-04,A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN,"['Cell Phones & Accessories', 'Cases, Holsters...",,"[, Elegani Butterfly Case 3D Pattern Back Cove...",,MinisDesign 3d Bling Crystal Bow Transparent C...,[],,ELEGANI,[Fashionable with unique 3D butterfly design f...,"[>#228,544 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,4.0,True,2014-02-03,A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN,"['Cell Phones & Accessories', 'Cases, Holsters...",,"[, Elegani Butterfly Case 3D Pattern Back Cove...",,MinisDesign 3d Bling Crystal Bow Transparent C...,[],,ELEGANI,[Fashionable with unique 3D butterfly design f...,"[>#228,544 in Cell Phones & Accessories (See T...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


# Merged Data Explore

In [17]:
dfTemp = df
display(str(dfTemp.shape))
dfDeDupe = dfTemp.drop_duplicates(subset=['reviewerID', 'asin', 'reviewText', 'reviewTime'],keep=False)
display(str(dfDeDupe.shape))

'(1128437, 12)'

'(1120076, 12)'

In [18]:
cat_One = dfMerge.groupby(['main_cat']).size().to_frame('reviewCnt')
cat_One = cat_One.sort_values(by='reviewCnt', ascending=False)        
display(cat_One.shape)    
# show top records
cat_One.head(29)

(29, 1)

,reviewCnt
main_cat,
Cell Phones & Accessories,1045517
All Electronics,41503
Computers,16199
Sports & Outdoors,3387
Home Audio & Theater,2683
Camera & Photo,2266
Amazon Home,2173
Car Electronics,1983
Portable Audio & Accessories,1960


In [19]:
# group by category
cat_Two = dfMerge.groupby(['category']).size().to_frame('reviewCnt')
cat_Two = cat_Two.sort_values(by='reviewCnt', ascending=False)        
display(cat_Two.shape)
# show top records
cat_Two.head(133)

(133, 1)

,reviewCnt
category,
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves', 'Basic Cases']",245197
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves']",152192
"['Cell Phones & Accessories', 'Accessories', 'Screen Protectors']",137476
"['Cell Phones & Accessories', 'Accessories', 'Cables']",63510
[],61861
...,...
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves', 'Imported', 'Demension:7.5x3.7x1.1 inchesLxWxT', 'Compatibility:Apple iphone 4 4s,iphone 5,5S,5C,iPhone 6plus,6,iphone 7plus,7,iphone 8, iphone 8plus,iphone x,Samsung Galaxy S3 s4,s6,s7,s8,galaxy note8,note6,note5,note4 and other cell phone.Great wallet and zipper purse.', 'Super convenient clutch bag and Wristlet purse for cellphone,wallet,purse and small stuffs.It can contain money and card slots.', 'It is made of premium leather which is durable and pretty.It is designed with fashion funk elements.', 'Great zipper bag,it is designed with seperated pouches for multiple cards and others .Best organizer for women.']",5
"['Cell Phones & Accessories', 'Accessories', 'Screen Protectors', 'The Skinomi TechSkin Motorola Moto 360 full body protector is specifically designed to provide complete full-body coverage; Material flexibility allows total coverage regardless of shape or size', 'Self-Healing, flexible, tough, military-grade thermoplastic urethane, designed to absorb impact; The TechSkin is resistant against scratches, punctures, UV light and will not yellow', 'Easy, error-proof, and anti-bubble installation; Liquid solution allows adjustments during installation; Significantly reduces dust, oil and fingerprint smudges', 'Clear full body skin protector for Motorola Moto 360 with smooth surface; A great alternative to a typical ""static cling"" protectors', 'Made in the USA and comes with the Skinomi Lifetime Replacement Warranty from Manufacturer']",5
"['Cell Phones & Accessories', 'Cases, Holsters & Sleeves', 'Elegant faux leather tapestry complimented with a unique printed design', 'Designed to hold up to 2 credit cards plus your money along-side your phone', 'Self stand function allows for convenient, comfortable media viewing or face talk', 'Precise cut out have been made to allow you full access to all functions', 'Manetic closing flap keeps the cover fimly closed for maximum screen protection']",5


In [20]:
# group by category
cat_Three = dfMerge.groupby(['title']).size().to_frame('reviewCnt')
cat_Three = cat_Three.sort_values(by='reviewCnt', ascending=False)        
display(cat_Three.shape)
# show top 5 records
cat_Three.head(5)

(47411, 1)

,reviewCnt
title,
Anker Astro E1 5200mAh Candy bar-Sized Ultra Compact Portable Charger (External Battery Power Bank) with High-Speed Charging PowerIQ Technology (Black),3019
"Anker 24W Dual USB Car Charger, PowerDrive 2 for iPhone Xs/XS Max/XR/X / 8/7 / 6 / Plus, iPad Pro/Air 2 / Mini, Note 5/4, LG, Nexus, HTC, and More",1684
"Anker PowerCore+ Mini, 3350mAh Lipstick-Sized Portable Charger (Premium Aluminum Power Bank), One of The Most Compact External Batteries, Compatible with iPhone Xs/XR, Android Smartphones and More",1634
"ZeroLemon Y313 Lightning to USB Plastic PVC Cable for Iphone 5s / 5c / 5, Ipad Air / Mini / Mini2, Ipad 4th Generation, Ipod 5th Generation, and Ipod Nano 7th Generation (PVC Green, 6-Inch)",1623
"Anker PowerCore 20100 - Ultra High Capacity Power Bank with 4.8A Output, PowerIQ Technology for iPhone, iPad and Samsung Galaxy and More (Black)",1514


In [21]:
# group by category, count distinct subreddits and posts
cat_All = dfMerge.groupby(['main_cat','category', 'title']).size().to_frame('reviewCnt')
cat_All = cat_All.sort_values(by='reviewCnt', ascending=False)
cat_All = cat_All.reset_index()
display(cat_All.shape)    
# show top 5 records
cat_All.head(5)

(47506, 4)

,main_cat,category,title,reviewCnt
0,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...",Anker Astro E1 5200mAh Candy bar-Sized Ultra C...,3019
1,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...","Anker 24W Dual USB Car Charger, PowerDrive 2 f...",1684
2,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...","Anker PowerCore+ Mini, 3350mAh Lipstick-Sized ...",1634
3,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...",ZeroLemon Y313 Lightning to USB Plastic PVC Ca...,1623
4,Cell Phones & Accessories,"['Cell Phones & Accessories', 'Accessories', '...",Anker PowerCore 20100 - Ultra High Capacity Po...,1514


In [22]:
cat_All['category'].nunique()

133

In [23]:
#Trim and fix column Dtypes for saving to database
value_list=["['Cell Phones & Accessories', 'Cell Phones']",
           "['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']",
           "['Cell Phones & Accessories', 'Cell Phones', 'Carrier Cell Phones']" 
           ]
#Trim records
dfTrim = dfMerge.loc[(dfMerge.category.isin(value_list))]
#dfTrim = dfTrim.drop(dfTrim.columns[5], axis=1) #drop the Style column
dfTrim.reset_index(inplace=True, drop=True)

#change the review to a date time
dfTrim['reviewTime'] = pd.to_datetime(dfTrim['reviewTime'], format="%m %d, %Y")

#Convert all columns that are objects to string.
#having an issue writing it to the database with type issues. 
#objectColumns = list(dfTrim.select_dtypes(['object']))
#dfTrim[objectColumns] = dfTrim[objectColumns].astype("string")

dfTrim.info()                    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63920 entries, 0 to 63919
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   overall          63920 non-null  float64       
 1   verified         63920 non-null  bool          
 2   reviewTime       63920 non-null  datetime64[ns]
 3   reviewerID       63920 non-null  object        
 4   asin             63920 non-null  object        
 5   style            29860 non-null  object        
 6   reviewerName     63912 non-null  object        
 7   reviewText       63887 non-null  object        
 8   summary          63903 non-null  object        
 9   unixReviewTime   63920 non-null  int64         
 10  vote             12512 non-null  object        
 11  image            616 non-null    object        
 12  category         63920 non-null  string        
 13  tech1            63920 non-null  object        
 14  description      63920 non-null  objec

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [24]:
dfTrim.head(5)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,imageURL,imageURLHighRes
0,1.0,False,2004-09-21,A5JLAU2ARJ0BO,B0000E3GWH,NaN,Old-and-Wise,This phone is ugly and heavy and has a terribl...,Bad reception,1095724800,2,NaN,"['Cell Phones & Accessories', 'Cell Phones', '...",,"[Compatible with T-Mobile cellular service, Sa...",,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,[],,Samsung,[],"[>#3,059,436 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,1.0,False,2004-09-21,A5JLAU2ARJ0BO,B0000E3GWH,NaN,Old-and-Wise,This phone is ugly and heavy and has a terribl...,Bad reception,1095724800,2,NaN,"['Cell Phones & Accessories', 'Cell Phones', '...",,"[Compatible with T-Mobile cellular service, Sa...",,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,[],,Samsung,[],"[>#3,059,436 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,4.0,False,2004-07-24,A1ACM1CBGORBN1,B0000E3GWH,NaN,Sherry Berry,I had the Samsung V205 and then I decided to t...,Pretty good phone.. an improvement from my v205,1090627200,15,NaN,"['Cell Phones & Accessories', 'Cell Phones', '...",,"[Compatible with T-Mobile cellular service, Sa...",,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,[],,Samsung,[],"[>#3,059,436 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,4.0,False,2004-07-24,A1ACM1CBGORBN1,B0000E3GWH,NaN,Sherry Berry,I had the Samsung V205 and then I decided to t...,Pretty good phone.. an improvement from my v205,1090627200,15,NaN,"['Cell Phones & Accessories', 'Cell Phones', '...",,"[Compatible with T-Mobile cellular service, Sa...",,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,[],,Samsung,[],"[>#3,059,436 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,3.0,False,2004-07-06,A2V48Q03FZUOSD,B0000E3GWH,NaN,Benjamin David,This is the first Samsung phone I have had and...,Not as user friendly as the Motorola V300-,1089072000,11,NaN,"['Cell Phones & Accessories', 'Cell Phones', '...",,"[Compatible with T-Mobile cellular service, Sa...",,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,[],,Samsung,[],"[>#3,059,436 in Cell Phones & Accessories (See...",[],{},Cell Phones & Accessories,,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


# Export option exploration

In [25]:
#import sqlite3

#con = sqlite3.connect('trimmed_cellphone.db')
#dfTrim.to_sql('src_data', con=con, if_exists='replace')
#con.commit()
#con.close()
#DB Size is 2.14GB

In [26]:
#dfTrim.to_csv('trimmed_cellphone.csv.gz', compression='gzip')
#GZip csv is 224MB

In [27]:
#dfTrim.to_pickle('trimmed_cellphone.pkl')
#296.4MB

# Data Cleaning

In [28]:
pd.set_option('mode.chained_assignment', None)
dfTrim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63920 entries, 0 to 63919
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   overall          63920 non-null  float64       
 1   verified         63920 non-null  bool          
 2   reviewTime       63920 non-null  datetime64[ns]
 3   reviewerID       63920 non-null  object        
 4   asin             63920 non-null  object        
 5   style            29860 non-null  object        
 6   reviewerName     63912 non-null  object        
 7   reviewText       63887 non-null  object        
 8   summary          63903 non-null  object        
 9   unixReviewTime   63920 non-null  int64         
 10  vote             12512 non-null  object        
 11  image            616 non-null    object        
 12  category         63920 non-null  string        
 13  tech1            63920 non-null  object        
 14  description      63920 non-null  objec

In [29]:
#Step 1 - Add in column for GUID
import uuid
dfTrim['uuid'] = [uuid.uuid4() for _ in range(len(dfTrim.index))]

dfTrim.uuid.duplicated().sum()

0

In [30]:
#Step 2 - Add in year column
dfTrim['reviewYear'] = dfTrim['reviewTime'].dt.year

In [33]:
#Step 3 - Trim to only chosen columns
columns = ['uuid', 'overall', 'verified', 'reviewTime', 'reviewYear',
            'reviewerID', 'asin', 'style', 'reviewText',
            'summary', 'vote','category',  'description',
            'title', 'brand', 'feature',  'main_cat','price']
dfExport = dfTrim[columns]
dfExport.info()
display(dfTrim.shape)
dfExport.drop_duplicates
display(dfTrim.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63920 entries, 0 to 63919
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uuid         63920 non-null  object        
 1   overall      63920 non-null  float64       
 2   verified     63920 non-null  bool          
 3   reviewTime   63920 non-null  datetime64[ns]
 4   reviewYear   63920 non-null  int64         
 5   reviewerID   63920 non-null  object        
 6   asin         63920 non-null  object        
 7   style        29860 non-null  object        
 8   reviewText   63887 non-null  object        
 9   summary      63903 non-null  object        
 10  vote         12512 non-null  object        
 11  category     63920 non-null  string        
 12  description  63920 non-null  object        
 13  title        63920 non-null  object        
 14  brand        63920 non-null  object        
 15  feature      63920 non-null  object        
 16  main

(63920, 32)

(63920, 32)

In [34]:
#Step 4 - Remove identified data we don't want
#identified in python book 02 during data column explore
value_list=["Cell Phones & Accessories"] 
#Trim records
dfExport = dfExport.loc[(dfExport.main_cat.isin(value_list))]
dfExport.reset_index(inplace=True, drop=True)
dfExport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63461 entries, 0 to 63460
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uuid         63461 non-null  object        
 1   overall      63461 non-null  float64       
 2   verified     63461 non-null  bool          
 3   reviewTime   63461 non-null  datetime64[ns]
 4   reviewYear   63461 non-null  int64         
 5   reviewerID   63461 non-null  object        
 6   asin         63461 non-null  object        
 7   style        29715 non-null  object        
 8   reviewText   63428 non-null  object        
 9   summary      63445 non-null  object        
 10  vote         12355 non-null  object        
 11  category     63461 non-null  string        
 12  description  63461 non-null  object        
 13  title        63461 non-null  object        
 14  brand        63461 non-null  object        
 15  feature      63461 non-null  object        
 16  main

In [35]:
#Step 5 - Drop null values

#drop all nulls in reviewText
dfExport = dfExport.dropna(subset=['reviewText']) 

#drop all nulls in "overall". Shouldn't be null but in case data changes
dfExport = dfExport.dropna(subset=['overall']) 

#drop all nulls in "summary". Shouldn't be null but in case data changes
dfExport = dfExport.dropna(subset=['summary']) 

dfExport.reset_index(drop=True, inplace=True)

dfExport.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63413 entries, 0 to 63412
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uuid         63413 non-null  object        
 1   overall      63413 non-null  float64       
 2   verified     63413 non-null  bool          
 3   reviewTime   63413 non-null  datetime64[ns]
 4   reviewYear   63413 non-null  int64         
 5   reviewerID   63413 non-null  object        
 6   asin         63413 non-null  object        
 7   style        29690 non-null  object        
 8   reviewText   63413 non-null  object        
 9   summary      63413 non-null  object        
 10  vote         12349 non-null  object        
 11  category     63413 non-null  string        
 12  description  63413 non-null  object        
 13  title        63413 non-null  object        
 14  brand        63413 non-null  object        
 15  feature      63413 non-null  object        
 16  main

In [38]:
#Step 10 - Create separate dataframes for each column for NLP analysis
dfExportReviewText = dfExport[['uuid', 'reviewText', 'overall']].copy()
dfExportSummary = dfExport[['uuid', 'summary', 'overall']].copy()
dfExportTitle = dfExport[['uuid', 'title', 'overall']].copy()

In [36]:
jarvis.showProjectDataFiles()

Here are all your project data files
[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project
----->** No files **

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/01_original
---[   gz][ json]--> Cell_Phones_and_Accessories_5.json.gz (161.24 MB)
---[   gz][ json]--> meta_Cell_Phones_and_Accessories.json.gz (343.33 MB)

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/02_working
[*][  pkl]---------> 02_Cellphone.pkl (46.32 MB)
[*][  pkl]---------> 02_NLP_ReviewTextData.pkl (87.00 MB)
[*][  pkl]---------> 02_NLP_SummaryData.pkl (8.32 MB)
[*][  pkl]---------> 02_NLP_TitleData.pkl (16.71 MB)
[*][  pkl]---------> 03_Cellphone.pkl (46.31 MB)
[*][  pkl]---------> 03_NLP_ReviewTextData.pkl (28.94 MB)
[*][  pkl]---------> 03_NLP_ReviewText_Narrow.pkl (17.13 MB)
[*][  pkl]---------> 03_NLP_SummaryData.pkl (3.82 MB)
[*][  pkl]---------> 03_NLP_TitleData.pkl (2.73 MB)
[*][  pkl]---------> 04_NLP_ReviewText_Narrow.pkl (16.95 MB)
[*][  pkl]---

In [39]:
dfExport.to_pickle(jarvis.DATA_DIR + '/02_working/01_Cellphone_small.pkl', protocol=4)
dfExportReviewText.to_pickle(jarvis.DATA_DIR + '/02_working/01_NLP_ReviewText_small.pkl', protocol=4)
dfExportSummary.to_pickle(jarvis.DATA_DIR + '/02_working/01_NLP_Summary_small.pkl', protocol=4)
dfExportTitle.to_pickle(jarvis.DATA_DIR + '/02_working/01_NLP_Title_small.pkl', protocol=4)